## Imports

In [52]:
import json
import pandas as pd
from pandas.io.json import json_normalize

---
## Reading and Checking the Data
First, let's read in the data, check the shape, and see if there are duplicate entries for any of the listed projects. For the first part, we'll read the json data directly into a dataframe.

In [53]:
wb_projects = pd.read_json('data/world_bank_projects.json')
print('(Rows, Columns): {}\n'.format(wb_projects.shape))
print(wb_projects.info())

(Rows, Columns): (500, 50)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500 entries, 0 to 499
Data columns (total 50 columns):
_id                         500 non-null object
approvalfy                  500 non-null int64
board_approval_month        500 non-null object
boardapprovaldate           500 non-null object
borrower                    485 non-null object
closingdate                 370 non-null object
country_namecode            500 non-null object
countrycode                 500 non-null object
countryname                 500 non-null object
countryshortname            500 non-null object
docty                       446 non-null object
envassesmentcategorycode    430 non-null object
grantamt                    500 non-null int64
ibrdcommamt                 500 non-null int64
id                          500 non-null object
idacommamt                  500 non-null int64
impagency                   472 non-null object
lendinginstr                495 non-null object
lending

In [54]:
print('Number of unique projects: {}'.format(wb_projects['project_name'].unique().shape[0]))

Number of unique projects: 500


Great! There are no duplicate projects so grouping will be easy.

---
## 1. Find the 10 Countries with the Most Projects
We'll use the `countryname` column for grouping since it's the most descriptive. To solve this we'll do the following:
1. group by country
2. aggregate by count
3. sort values descending
4. limit top ten  

We can accomplish this with a single command using chained methods in pandas.

In [67]:
top_ten_countries = wb_projects.groupby('countryname')\
                               .count()[['id']]\
                               .sort_values(by='id', ascending=False)\
                               .head(10)
top_ten_countries.columns = ['Total Projects']
top_ten_countries.index.name = 'Country Name'
top_ten_countries

,Total Projects
Country Name,
People's Republic of China,19
Republic of Indonesia,19
Socialist Republic of Vietnam,17
Republic of India,16
Republic of Yemen,13
People's Republic of Bangladesh,12
Nepal,12
Kingdom of Morocco,12
Republic of Mozambique,11


In [70]:
print('Number of countries: {}'.format(wb_projects['countryname'].unique().shape[0]))

Number of countries: 118


Interestingly, Africa (a continent, not a country) comes in tied at \#9 with Mozambique, which is a country in East Africa. This brings a few questions to mind:

1. Do the projects have different geographic scopes (i.e. state, country, continent)?
2. Were the project details entered incorrectly (missing country name)?

Further investigation would be needed.

---
## 2. Find the Top 10 Major Project Themes (using column 'mjtheme_namecode')
For this exercise, we'll use the `json_normalize` function from the `pandas.io.json` module. This is because the `mjtheme_namecode` column consists of lists of dictionaries pertaining to the theme(s) of each project. Some projects have multiple themes, hence the multiple entries.

In [29]:
theme_string = json.load((open('data/world_bank_projects.json')))
theme_df = json_normalize(theme_string, 'mjtheme_namecode')

print(theme_df.shape)
display(theme_df.head())

(1499, 2)


,code,name
0,8,Human development
1,11,
2,1,Economic management
3,6,Social protection and risk management
4,5,Trade and integration


We can see here that the name is missing for for the second row of `theme_df`. We'll go ahead and solve exercise 3 now, and then finish off exercise 2 once that's done.

---
## 3. Fill in the Missing Theme Names
For this we'll create a dictionary of codes and names to be used as a lookup table for filling in missing names.

In [45]:
theme_dict = theme_df.groupby('code').max().to_dict()['name']
theme_dict

{'1': 'Economic management',
 '10': 'Rural development',
 '11': 'Environment and natural resources management',
 '2': 'Public sector governance',
 '3': 'Rule of law',
 '4': 'Financial and private sector development',
 '5': 'Trade and integration',
 '6': 'Social protection and risk management',
 '7': 'Social dev/gender/inclusion',
 '8': 'Human development',
 '9': 'Urban development'}

Now we can fill the missing theme names.

In [66]:
#fill missing names using the key (code), value (name) pairs in theme_dict
for row in theme_df.iterrows(): row[1]['name'] = theme_dict[row[1]['code']]

In [50]:
theme_df.head()

,code,name
0,8,Human development
1,11,Environment and natural resources management
2,1,Economic management
3,6,Social protection and risk management
4,5,Trade and integration


The missing entry has been correctly filled.

Now to finish of exercise 2, we'll count the number of occurences of each theme.

In [73]:
#need to add another column to hold the total counts for the grouping
theme_df['totals'] = 0

#now perform the grouping, aggregate, sort, and limit
top_ten_themes = theme_df.groupby(['code', 'name'])\
                         .count()\
                         .sort_values(by='totals', ascending=False)\
                         .head(10)
top_ten_themes

,,totals
code,name,
11,Environment and natural resources management,250
10,Rural development,216
8,Human development,210
2,Public sector governance,199
6,Social protection and risk management,168
4,Financial and private sector development,146
7,Social dev/gender/inclusion,130
5,Trade and integration,77
9,Urban development,50


These are the top 10 project themes (out of 11). 